# Coeficiente de Arrastre con GPLearn

In [1]:
import pandas as pd
import matplotlib 
from gplearn.genetic import SymbolicRegressor
from gplearn.functions import make_function
from gplearn.fitness import make_fitness
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.utils.random import check_random_state
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error, r2_score
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np
#import graphviz
import pandas as pd
import random
import re
from sklearn.preprocessing import MinMaxScaler

### Cargando Datos

In [2]:
# cargando datos

# importacion de DF
file_path = 'D:\\CODES\\LIB_SR\\Dataset_coeficientes\\df_cdrag_25.txt'
df_cdrag_25 = pd.read_csv(file_path, delimiter=',')

file_path = 'D:\\CODES\\LIB_SR\\Dataset_coeficientes\\df_cdrag_53.txt'
df_cdrag_53 = pd.read_csv(file_path, delimiter=',')

file_path = 'D:\\CODES\\LIB_SR\\Dataset_coeficientes\\df_cdrag_74.txt'
df_cdrag_74 = pd.read_csv(file_path, delimiter=',')

file_path = 'D:\\CODES\\LIB_SR\\Dataset_coeficientes\\df_cdrag_102.txt'
df_cdrag_102 = pd.read_csv(file_path, delimiter=',')

In [3]:
# definiendo conjuntos de train y test

# definiendo conjunto de train
df_cdrag_train = pd.concat([df_cdrag_25, df_cdrag_74], ignore_index=True)

# separando entre x e y
y_train = df_cdrag_train.drop(columns=['Current','K','Flujo','t_viento','Diametro','col_fluido','col_celda','n_fluido','n_celda','Rem','colIndex'])
X_train = df_cdrag_train.drop(columns=['Current','Flujo','t_viento','Diametro','col_fluido','col_celda','n_fluido',
                                    'n_celda','colIndex','cdrag'])


# definiendo conjunto de test

y_test = df_cdrag_53.drop(columns=['Current','K','Flujo','t_viento','Diametro','col_fluido','col_celda','n_fluido','n_celda','Rem','colIndex'])
X_test = df_cdrag_53.drop(columns=['Current','Flujo','t_viento','Diametro','col_fluido','col_celda','n_fluido',
                                    'n_celda','colIndex','cdrag'])

y_train = y_train.values.ravel()

### Experimento: incluyendo coeficientes de Rafael

In [4]:
# creando funciones auxiliares

# S ^ (-0.6)
def pot1(x1):
    with np.errstate(divide='ignore', invalid='ignore'):
        result = np.where(x1 > 0, np.power(x1, -0.6), 0)
    return result

pot1_fn = make_function(function=pot1, 
                        name='pot1', 
                        arity=1)

# 5*Re ^ (-0.23)
def pot2(x1):
    with np.errstate(divide='ignore', invalid='ignore'):
        result = np.where(x1 > 0, 5*np.power(x1, -0.23), 0)
    return result

pot2_fn = make_function(function=pot2, 
                        name='pot2', 
                        arity=1)



In [48]:
def rewrite(expression):

    # Reemplazar funciones específicas con sus representaciones matemáticas
    expression = re.sub(r"add\(([^,]+), ([^\)]+)\)", r"(\1 + \2)", expression)
    expression = re.sub(r"sub\(([^,]+), ([^\)]+)\)", r"(\1 - \2)", expression)
    expression = re.sub(r"mul\(([^,]+), ([^\)]+)\)", r"(\1 * \2)", expression)
    expression = re.sub(r"div\(([^,]+), ([^\)]+)\)", r"(\1 / \2)", expression)
    expression = re.sub(r"pow\(([^,]+), ([^\)]+)\)", r"\1 ** \2", expression)
    
    # Reemplazar la función pot1
    expression = re.sub(r"pot1\((X\d)\)", r"\1 ** (-0.6)", expression)
    # Reemplazar la función pot2
    expression = re.sub(r"pot2\((X\d)\)", r"5 * \1 ** (-0.23)", expression)
    
    # Reemplazar variables (ajustar según el modelo)
    expression = expression.replace("X0", "S")
    expression = expression.replace("X1", "Re")
    
    return expression


In [5]:
# entrenando modelo 

# Modelo
model = SymbolicRegressor(population_size=5000,
                        generations=20, 
                        stopping_criteria=0.01,
                        p_crossover=0.7, 
                        p_subtree_mutation=0.1,
                        p_hoist_mutation=0.05, 
                        p_point_mutation=0.1,
                        max_samples=0.9, 
                        verbose=1,
                        parsimony_coefficient=0.01, 
                        random_state=123,
                        function_set=['add', 'sub', 'mul', 'div', 'sqrt', 'log',pot1_fn,pot2_fn])

model.fit(X_train, y_train)

# Obtener la expresión simbólica
expression = model._program
#print("Expresión:", expression)


d:\env\lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0    13.80      4.44235e+07        5        0.0727931        0.0695565     12.12m
   1     9.11      3.59451e+07        5        0.0723137        0.0738714      8.17m
   2     6.64          25901.6        5        0.0720671        0.0760905     10.96m
   3     6.11          12859.9        9        0.0698535        0.0740827      8.44m
   4     6.26          37314.4        5        0.0718866        0.0777156      9.97m
   5     5.32          22445.9        8        0.0717459        0.0728612      6.35m
   6     5.20          23145.1        5        0.0717727        0.0787408      7.15m
   7     5.17      1.83698e+07        5         0.071877         0.077802      6.56m
   8     5.18          22152.1        5        0.0718211        0.0783049  

In [49]:
rewrite(expression.__str__())

'(5 * Re ** (-0.23) + S ** (-0.6))'

In [6]:
# Predecir en los datos de prueba
y_pred = model.predict(X_test)


In [10]:
# comparar con modelo de Rafael

def cdrag(S, Re):
    return S**(-0.6) + 5*Re**(-0.23)

def cdrag_gp(S, Re):
    return S**(-0.6) + 5*Re**(-0.23)

In [11]:
# Calcular cdrag_pred y cdrag_gp_pred en vectores separados
cdrag_pred = X_test.apply(lambda row: cdrag(row['K'], row['Rem']), axis=1).values
cdrag_gp_pred = X_test.apply(lambda row: cdrag_gp(row['K'], row['Rem']), axis=1).values

# Calcular R^2 y MSE para cdrag
r2_cdrag = r2_score(y_test['cdrag'], cdrag_pred)
mse_cdrag = mean_squared_error(y_test['cdrag'], cdrag_pred)

# Calcular MAE y MAPE para cdrag
mae_cdrag = mean_absolute_error(y_test['cdrag'], cdrag_pred)
mape_cdrag = mean_absolute_percentage_error(y_test['cdrag'], cdrag_pred)

# Calcular R^2 y MSE para cdrag_gp
r2_cdrag_gp = r2_score(y_test['cdrag'], cdrag_gp_pred)
mse_cdrag_gp = mean_squared_error(y_test['cdrag'], cdrag_gp_pred)

# Calcular MAE y MAPE para cdrag_gp
mae_cdrag_gp = mean_absolute_error(y_test['cdrag'], cdrag_gp_pred)
mape_cdrag_gp = mean_absolute_percentage_error(y_test['cdrag'], cdrag_gp_pred)

#  Mostrar Metricas 

print(f"R^2 cdrag original: {r2_cdrag}, R^2 cdrag GPLearn: {r2_cdrag_gp} ")
print(f"MSE cdrag original: {mse_cdrag}, MSE cdrag GPLearn: {mse_cdrag_gp} ")
print(f"MAE cdrag original: {mae_cdrag}, MAE cdrag GPLearn: {mae_cdrag_gp} ")
print(f"MAPE cdrag original: {mape_cdrag*100}*100, MAPE cdrag GPLearn: {mape_cdrag_gp*100}*100 ")

R^2 cdrag original: 0.8904926780136748, R^2 cdrag GPLearn: 0.8904926780136748 
MSE cdrag original: 0.011588230963563853, MSE cdrag GPLearn: 0.011588230963563853 
MAE cdrag original: 0.07580331737181549, MAE cdrag GPLearn: 0.07580331737181549 
MAPE cdrag original: 0.03744829246296146, MAPE cdrag GPLearn: 0.03744829246296146 


### Prueba 2: Separacion entre laminar y turbulento

In [34]:
df_cdrag_train = pd.concat([df_cdrag_25, df_cdrag_74], ignore_index=True)

# separando entre desplazamiento laminar y turbulento
df_cdrag_train_lam = df_cdrag_train[df_cdrag_train['Rem'] <= 2000]
df_cdrag_train_turb = df_cdrag_train[df_cdrag_train['Rem'] > 2000]

y_train_lam = df_cdrag_train_lam.drop(columns=['Current','K','Flujo','t_viento','Diametro','col_fluido','col_celda','n_fluido','n_celda','Rem','colIndex'])
X_train_lam = df_cdrag_train_lam.drop(columns=['Current','Flujo','t_viento','Diametro','col_fluido','col_celda','n_fluido',
                                    'n_celda','colIndex','cdrag'])

y_train_turb = df_cdrag_train_turb.drop(columns=['Current','K','Flujo','t_viento','Diametro','col_fluido','col_celda','n_fluido','n_celda','Rem','colIndex'])
X_train_turb = df_cdrag_train_turb.drop(columns=['Current','Flujo','t_viento','Diametro','col_fluido','col_celda','n_fluido',
                                    'n_celda','colIndex','cdrag'])

test_lam = df_cdrag_53[df_cdrag_53['Rem'] <= 2000]
test_turb = df_cdrag_53[df_cdrag_53['Rem'] > 2000]

# definiendo conjunto de test

y_test_lam = test_lam.drop(columns=['Current','K','Flujo','t_viento','Diametro','col_fluido','col_celda','n_fluido','n_celda','Rem','colIndex'])
X_test_lam = test_lam.drop(columns=['Current','Flujo','t_viento','Diametro','col_fluido','col_celda','n_fluido',
                                    'n_celda','colIndex','cdrag'])

y_test_turb = test_turb.drop(columns=['Current','K','Flujo','t_viento','Diametro','col_fluido','col_celda','n_fluido','n_celda','Rem','colIndex'])
X_test_turb = test_turb.drop(columns=['Current','Flujo','t_viento','Diametro','col_fluido','col_celda','n_fluido',
                                    'n_celda','colIndex','cdrag'])


In [38]:
# S ^ (-0.6)
def pot1(x1):
    with np.errstate(divide='ignore', invalid='ignore'):
        result = np.where(x1 > 0, np.power(x1, -0.6), 0)
    return result

pot1_fn = make_function(function=pot1, 
                        name='pot1', 
                        arity=1)

# 5*Re ^ (-0.23)
def pot2(x1):
    with np.errstate(divide='ignore', invalid='ignore'):
        result = np.where(x1 > 0, 5*np.power(x1, -0.23), 0)
    return result

pot2_fn = make_function(function=pot2, 
                        name='pot2', 
                        arity=1)


In [40]:
# entrenando modelo laminar

# Modelo
model = SymbolicRegressor(population_size=5000,
                           generations=25, 
                           stopping_criteria=0.01,
                           p_crossover=0.7, 
                           p_subtree_mutation=0.1,
                           p_hoist_mutation=0.05, 
                           p_point_mutation=0.1,
                           max_samples=0.9, 
                           verbose=1,
                           parsimony_coefficient=0.01, 
                           random_state=123,
                           function_set=['add', 'sub', 'mul', 'div',pot1_fn,pot2_fn],
                           const_range=(-10, 10))

model.fit(X_train_lam, y_train_lam)

# Obtener la expresión simbólica
expression = model._program
print("Expresión:", expression)

# Predecir en los datos de prueba
y_pred = model.predict(X_test_lam)

# Calcular métricas
mse = mean_squared_error(y_test_lam, y_pred)
r2 = r2_score(y_test_lam, y_pred)

print("MSE:", mse)
print("R^2:", r2)

d:\env\lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0    19.46      1.16325e+11       14        0.0989867         0.097607      9.94m
   1    11.90      3.47286e+06       25         0.098282          0.10394      5.99m
   2    10.16      1.99623e+06        9        0.0980644        0.0956614      6.94m
   3     9.37      1.76432e+09        8        0.0961371        0.0935922      6.71m
   4     7.70          11995.3       10          0.09412        0.0958618      5.18m
   5     5.28      2.56837e+06       12        0.0967676        0.0973654      5.82m
   6     5.26           265602        5         0.098622         0.113828      4.71m
   7     5.24          3651.69        6         0.096651        0.0977687      5.83m
   8     5.24          13647.6        4        0.0969893        0.0892327  

In [41]:
# entrenando modelo turbulento

# Modelo
model = SymbolicRegressor(population_size=5000,
                           generations=25, 
                           stopping_criteria=0.01,
                           p_crossover=0.7, 
                           p_subtree_mutation=0.1,
                           p_hoist_mutation=0.05, 
                           p_point_mutation=0.1,
                           max_samples=0.9, 
                           verbose=1,
                           parsimony_coefficient=0.01, 
                           random_state=123,
                           function_set=['add', 'sub', 'mul', 'div',pot1_fn,pot2_fn],
                           const_range=(-10, 10))

model.fit(X_train_turb, y_train_turb)

# Obtener la expresión simbólica
expression = model._program
print("Expresión:", expression)

# Predecir en los datos de prueba
y_pred = model.predict(X_test_turb)

# Calcular métricas
mse = mean_squared_error(y_test_turb, y_pred)
r2 = r2_score(y_test_turb, y_pred)

print("MSE:", mse)
print("R^2:", r2)

d:\env\lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0    19.46      8.02502e+12       13        0.0642244        0.0631852     26.53m
   1    11.50      5.34598e+07        5        0.0638259        0.0667703     27.34m
   2     9.58      4.75667e+07        5        0.0635333        0.0694023     28.93m
   3     6.43      4.82039e+07        8        0.0634863        0.0698258     23.54m
   4     5.83          70092.2        5        0.0635609        0.0691547     25.87m
   5     5.26      6.08778e+07        5         0.063477        0.0699095     13.39m
   6     5.26        3.352e+06        5        0.0635552        0.0692055     12.44m
   7     5.24          24680.8        5        0.0634513          0.07014      7.98m
   8     5.24           136728        5        0.0634564        0.0700944  

### Ajuste automatico de Exponencial

In [4]:

def pot(x1, const):
    con1 = x1 > 0
    con2 = (const >= -1) & (const < 0)
    with np.errstate(divide='ignore', invalid='ignore'):
        result = np.where(con1 & con2, np.power(x1,const), 0)
    return result

pot= make_function(function=pot, name='pot', arity=2)

# A * x**B
def pot2(x1):
    con1 = x1 > 0
    # con2 = (B >= -0.24) & (B <= -0.22)
    # con3 = (A >=4.9) & (A <=5.1)
    with np.errstate(divide='ignore', invalid='ignore'):
        result = np.where(con1  , 5 *np.power(x1,-0.23), 0)
    return result

pot2= make_function(function=pot2, name='pot2', arity=1)

In [5]:
# entrenando modelo 

# Modelo
model = SymbolicRegressor(population_size=5000,
                        generations=20, 
                        stopping_criteria=0.01,
                        p_crossover=0.7, 
                        p_subtree_mutation=0.1,
                        p_hoist_mutation=0.05, 
                        p_point_mutation=0.1,
                        max_samples=0.9, 
                        verbose=1,
                        parsimony_coefficient=0.01, 
                        random_state=42,
                        function_set=['add', 'sub', 'mul', 'div',pot, pot2])

model.fit(X_train, y_train)

# Obtener la expresión simbólica
expression = model._program
print("Expresión:", expression)

    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left


C:\Users\lucas\AppData\Local\Temp\ipykernel_9276\2036374849.py:5: RuntimeWarning: overflow encountered in power
  result = np.where(con1 & con2, np.power(x1,const), 0)


   0    27.82      5.19856e+12       14         0.119096         0.119238     38.36m
   1    15.19      4.99107e+09        6        0.0755688        0.0714117     25.62m
   2    14.27      3.69476e+07        6        0.0750084        0.0764555     23.72m
   3    10.62      1.86747e+07       14        0.0728228        0.0721428     13.91m
   4     8.56      1.33538e+11        6         0.072321        0.0766594     13.39m
   5     6.04          16224.3        6        0.0722672         0.077143      8.90m
   6     5.84      1.00035e+07        6        0.0721768         0.077957      7.79m
   7     6.30          30283.6        6        0.0721093        0.0785644      7.74m
   8     6.25          12617.2        6        0.0718201        0.0811672      5.08m
   9     6.22          37501.5        6        0.0721354          0.07833      4.51m
  10     6.28          46276.7        8        0.0719365        0.0737697      3.89m
  11     6.16      1.85982e+07        6        0.0720783        0

In [7]:
y_pred = model.predict(X_test)

# Calcular métricas
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("MSE:", mse)
print("R^2:", r2)

MSE: 0.01146079397831767
R^2: 0.8916969414616678


In [8]:
def cdrag(S, Re):
    return S**(-0.6) + 5*Re**(-0.23)

def cdrag_gp(S, Re):
    return S**(-0.606) + 5*Re**(-0.23)

In [9]:
# Calcular cdrag_pred y cdrag_gp_pred en vectores separados
cdrag_pred = X_test.apply(lambda row: cdrag(row['K'], row['Rem']), axis=1).values
cdrag_gp_pred = X_test.apply(lambda row: cdrag_gp(row['K'], row['Rem']), axis=1).values

# Calcular R^2 y MSE para cdrag
r2_cdrag = r2_score(y_test['cdrag'], cdrag_pred)
mse_cdrag = mean_squared_error(y_test['cdrag'], cdrag_pred)

# Calcular MAE y MAPE para cdrag
mae_cdrag = mean_absolute_error(y_test['cdrag'], cdrag_pred)
mape_cdrag = mean_absolute_percentage_error(y_test['cdrag'], cdrag_pred)

# Calcular R^2 y MSE para cdrag_gp
r2_cdrag_gp = r2_score(y_test['cdrag'], cdrag_gp_pred)
mse_cdrag_gp = mean_squared_error(y_test['cdrag'], cdrag_gp_pred)

# Calcular MAE y MAPE para cdrag_gp
mae_cdrag_gp = mean_absolute_error(y_test['cdrag'], cdrag_gp_pred)
mape_cdrag_gp = mean_absolute_percentage_error(y_test['cdrag'], cdrag_gp_pred)

#  Mostrar Metricas 

print(f"R^2 cdrag original: {r2_cdrag}, R^2 cdrag GPLearn: {r2_cdrag_gp} ")
print(f"MSE cdrag original: {mse_cdrag}, MSE cdrag GPLearn: {mse_cdrag_gp} ")
print(f"MAE cdrag original: {mae_cdrag}, MAE cdrag GPLearn: {mae_cdrag_gp} ")
print(f"MAPE cdrag original: {mape_cdrag*100}*100, MAPE cdrag GPLearn: {mape_cdrag_gp*100}*100 ")

R^2 cdrag original: 0.8904926780136748, R^2 cdrag GPLearn: 0.8917492597780703 
MSE cdrag original: 0.011588230963563853, MSE cdrag GPLearn: 0.011455257574695527 
MAE cdrag original: 0.07580331737181549, MAE cdrag GPLearn: 0.075295920219909 
MAPE cdrag original: 3.7448292462961463*100, MAPE cdrag GPLearn: 3.7237039210106486*100 


### Ajuste Automatico de constantes V1.5

In [ ]:
def pot(x1, const):
    con1 = x1 > 0
    con2 = (const >= -1) & (const < 0)
    with np.errstate(divide='ignore', invalid='ignore'):
        result = np.where(con1 & con2, np.power(x1,const), 0)
    return result

pot= make_function(function=pot, name='pot', arity=2)

# A * x**B
def pot2(x1):
    con1 = x1 > 0
    con2 = (B >= -1) & (B <= 0)
    # con3 = (A >=4.9) & (A <=5.1)
    with np.errstate(divide='ignore', invalid='ignore'):
        result = np.where(con1  & con2, 5 *np.power(x1,-0.2), 0)
    return result

pot2= make_function(function=pot2, name='pot2', arity=1)

### Ajuste automatico de constantes V2

In [ ]:
def rewrite(expression):

    # Reemplazar funciones específicas con sus representaciones matemáticas
    expression = re.sub(r"add\(([^,]+), ([^\)]+)\)", r"(\1 + \2)", expression)
    expression = re.sub(r"sub\(([^,]+), ([^\)]+)\)", r"(\1 - \2)", expression)
    expression = re.sub(r"mul\(([^,]+), ([^\)]+)\)", r"(\1 * \2)", expression)
    expression = re.sub(r"div\(([^,]+), ([^\)]+)\)", r"(\1 / \2)", expression)
    #expression = re.sub(r"pow\(([^,]+), ([^\)]+)\)", r"\1 ** \2", expression)
    
    # Reemplazar la función pot
    expression = re.sub(r"pot\((X\d)\)", r"\1 ** (-0.6)", expression)
    # Reemplazar la función pot2
    #expression = re.sub(r"pot2\((X\d)\)", r"5 * \1 ** (-0.23)", expression)
    
    # Reemplazar variables (ajustar según el modelo)
    expression = expression.replace("X0", "S")
    expression = expression.replace("X1", "Re")
    
    return expression

In [17]:
# A* x ^ B

def pot(x1, A, B):
    con1 = x1 > 0
    con2 = (B >= -1) & (B < 0)
    con3 = A >= 1
    with np.errstate(divide='ignore', invalid='ignore'):
        result = np.where(con1 & con2 & con3, A*np.power(x1,B), 0)
    return result

pot_fn= make_function(function=pot, name='pot', arity=3)

In [18]:
# Modelo
model = SymbolicRegressor(population_size=5000,
                        generations=20, 
                        stopping_criteria=0.01,
                        const_range = (-1.0, 10.0),
                        p_crossover=0.7, 
                        p_subtree_mutation=0.1,
                        p_hoist_mutation=0.05, 
                        p_point_mutation=0.1,
                        max_samples=0.9, 
                        verbose=1,
                        parsimony_coefficient=0.01, 
                        random_state=0,
                        function_set=['add',pot_fn])

model.fit(X_train, y_train)

# Obtener la expresión simbólica
expression = model._program
print("Expresión:", expression)

    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left


C:\Users\lucas\AppData\Local\Temp\ipykernel_2232\1152348812.py:8: RuntimeWarning: overflow encountered in power
  result = np.where(con1 & con2 & con3, A*np.power(x1,B), 0)
C:\Users\lucas\AppData\Local\Temp\ipykernel_2232\1152348812.py:8: RuntimeWarning: overflow encountered in multiply
  result = np.where(con1 & con2 & con3, A*np.power(x1,B), 0)


   0    90.51          1179.24       10         0.235563          0.24473     63.57m
   1    18.37          270.561       12         0.235133         0.248594     15.97m
   2    11.64          256.708       13         0.235097         0.248922      9.23m
   3     7.61          273.857       13         0.234758         0.251969      9.07m
   4     2.79          135.077        6         0.234767         0.251887      3.32m
   5     1.89          145.019        1         0.234883         0.250843      2.59m
   6     1.68           122.97        1         0.234159         0.257363      2.67m
   7     1.56          113.227        1         0.234534         0.253993      1.85m
   8     1.72              121        1         0.234586          0.25352      2.13m
   9     1.79          126.542        1         0.234487         0.254411      1.72m
  10     1.54          112.799        1         0.234258         0.256473      1.53m
  11     1.66          122.773        1         0.234233         

### Ajuste Automatico de Constantes V3

In [19]:
def pot(x1, const):
    con1 = x1 > 0
    con2 = (const >= -1) & (const < 0)
    with np.errstate(divide='ignore', invalid='ignore'):
        result = np.where(con1 & con2, np.power(x1,const), 0)
    return result

pot= make_function(function=pot, name='pot', arity=2)

# A * x**B
def pot2(x1,A,B):
    con1 = x1 > 0
    con2 = (B >= -1) & (B < 0)
    con3 = (A >=2) & (A <=8)
    with np.errstate(divide='ignore', invalid='ignore'):
        result = np.where(con1  , A *np.power(x1,B), 0)
    return result

pot2= make_function(function=pot2, name='pot2', arity=3)

In [21]:
# entrenando modelo 

# Modelo
model = SymbolicRegressor(population_size=5000,
                        generations=20, 
                        stopping_criteria=0.01,
                        const_range = (-1.0, 8.0),
                        p_crossover=0.7, 
                        p_subtree_mutation=0.1,
                        p_hoist_mutation=0.05, 
                        p_point_mutation=0.1,
                        max_samples=0.9, 
                        verbose=1,
                        parsimony_coefficient=0.01, 
                        random_state=0,
                        function_set=['add', 'sub', 'mul', 'div',pot, pot2])

model.fit(X_train, y_train)

# Obtener la expresión simbólica
expression = model._program
print("Expresión:", expression)

    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left


C:\Users\lucas\AppData\Local\Temp\ipykernel_2232\565715426.py:16: RuntimeWarning: overflow encountered in power
  result = np.where(con1  , A *np.power(x1,B), 0)
C:\Users\lucas\AppData\Local\Temp\ipykernel_2232\565715426.py:5: RuntimeWarning: overflow encountered in power
  result = np.where(con1 & con2, np.power(x1,const), 0)
d:\env\lib\site-packages\gplearn\functions.py:46: RuntimeWarning: invalid value encountered in multiply
  return self.function(*args)
d:\env\lib\site-packages\numpy\lib\_function_base_impl.py:573: RuntimeWarning: invalid value encountered in multiply
  avg = avg_as_array = np.multiply(a, wgt,
C:\Users\lucas\AppData\Local\Temp\ipykernel_2232\565715426.py:16: RuntimeWarning: overflow encountered in multiply
  result = np.where(con1  , A *np.power(x1,B), 0)
d:\env\lib\site-packages\gplearn\functions.py:127: RuntimeWarning: overflow encountered in divide
  return np.where(np.abs(x2) > 0.001, np.divide(x1, x2), 1.)
d:\env\lib\site-packages\gplearn\functions.py:46: Run

   0    56.37              nan       41              nan              nan     47.32m
   1    63.94              nan      185              nan              nan     45.86m
   2    72.62              nan       25              nan              nan     46.33m
   3    80.54              nan       18              nan              nan     43.88m
   4    86.16              nan       63              nan              nan     43.39m
   5    90.19              nan       10              nan              nan     48.65m


d:\env\lib\site-packages\gplearn\functions.py:46: RuntimeWarning: overflow encountered in subtract
  return self.function(*args)


   6    94.50              nan       80              nan              nan     42.28m
   7    97.62              nan       41              nan              nan     39.85m
   8   103.19              nan       22              nan              nan     43.79m
   9   106.83              nan      213              nan              nan     53.54m
  10   113.59              nan      180              nan              nan     35.03m
  11   118.24              nan       47              nan              nan     31.27m
  12   124.56              nan       22              nan              nan     31.13m
  13   129.84              nan      161              nan              nan     22.44m
  14   134.74              nan      165              nan              nan     23.84m


KeyboardInterrupt: 

### Ajuste automatico de toda la funcion

In [11]:
# x1^A + B*x2^C

def pot(x1, x2,A,B,C):
    con1 = (x1 > 0)
    con2 = (x2 > 0)
    con3 = (A >= -1) & (A < 0)
    con4 = (B > 1)
    con5 = (A >= -1) & (A < 0)
    with np.errstate(divide='ignore', invalid='ignore'):
        result = np.where(con1 & con2 & con3 & con4 & con5, np.power(x1,A) + B*np.power(x2,C), 0)
    return result

pot= make_function(function=pot, name='pot', arity=5)

In [12]:
# Modelo
model = SymbolicRegressor(population_size=5000,
                        generations=20, 
                        stopping_criteria=0.01,
                        const_range = (-1.0, 10.0),
                        p_crossover=0.7, 
                        p_subtree_mutation=0.1,
                        p_hoist_mutation=0.05, 
                        p_point_mutation=0.1,
                        max_samples=0.9, 
                        verbose=1,
                        parsimony_coefficient=0.01, 
                        random_state=0,
                        function_set=[pot])

model.fit(X_train, y_train)

# Obtener la expresión simbólica
expression = model._program
print("Expresión:", expression)

    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left


C:\Users\lucas\AppData\Local\Temp\ipykernel_2232\1665499163.py:10: RuntimeWarning: overflow encountered in power
  result = np.where(con1 & con2 & con3 & con4 & con5, np.power(x1,A) + B*np.power(x2,C), 0)
C:\Users\lucas\AppData\Local\Temp\ipykernel_2232\1665499163.py:10: RuntimeWarning: overflow encountered in multiply
  result = np.where(con1 & con2 & con3 & con4 & con5, np.power(x1,A) + B*np.power(x2,C), 0)
C:\Users\lucas\AppData\Local\Temp\ipykernel_2232\1665499163.py:10: RuntimeWarning: overflow encountered in add
  result = np.where(con1 & con2 & con3 & con4 & con5, np.power(x1,A) + B*np.power(x2,C), 0)


   0  2815.54          2.57224      631         0.938607         0.969947   3720.94m


d:\env\lib\site-packages\numpy\lib\_function_base_impl.py:573: RuntimeWarning: invalid value encountered in multiply
  avg = avg_as_array = np.multiply(a, wgt,


   1    21.82              nan        6              nan              nan     30.28m
   2     5.46              nan        6              nan              nan      9.70m


d:\env\lib\site-packages\numpy\_core\_methods.py:52: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)


   3     2.44              nan        6              nan              nan      6.39m
   4     6.24              nan        6              nan              nan      9.74m
   5     7.67              nan       11              nan              nan     10.96m
   6     8.45              nan       11              nan              nan     10.30m
   7     9.97              nan       11              nan              nan     10.41m
   8    10.77              nan       11              nan              nan     10.77m
   9    11.86              nan       16              nan              nan     10.37m
  10    12.90              nan        6              nan              nan      9.66m
  11    14.39              nan       21              nan              nan      9.18m
  12    15.51              nan       36              nan              nan     10.77m
  13    16.83              nan       11              nan              nan     11.57m
  14    18.13              nan       11              nan         

In [15]:
def cdrag(S, Re):
    return S**(-0.6) + 5*Re**(-0.23)

def cdrag_gp(S, Re):
    return S**(-0.056) + S*(6.450**Re)

In [16]:
# Calcular cdrag_pred y cdrag_gp_pred en vectores separados
cdrag_pred = X_test.apply(lambda row: cdrag(row['K'], row['Rem']), axis=1).values
cdrag_gp_pred = X_test.apply(lambda row: cdrag_gp(row['K'], row['Rem']), axis=1).values

# Calcular R^2 y MSE para cdrag
r2_cdrag = r2_score(y_test['cdrag'], cdrag_pred)
mse_cdrag = mean_squared_error(y_test['cdrag'], cdrag_pred)

# Calcular MAE y MAPE para cdrag
mae_cdrag = mean_absolute_error(y_test['cdrag'], cdrag_pred)
mape_cdrag = mean_absolute_percentage_error(y_test['cdrag'], cdrag_pred)

# Calcular R^2 y MSE para cdrag_gp
r2_cdrag_gp = r2_score(y_test['cdrag'], cdrag_gp_pred)
mse_cdrag_gp = mean_squared_error(y_test['cdrag'], cdrag_gp_pred)

# Calcular MAE y MAPE para cdrag_gp
mae_cdrag_gp = mean_absolute_error(y_test['cdrag'], cdrag_gp_pred)
mape_cdrag_gp = mean_absolute_percentage_error(y_test['cdrag'], cdrag_gp_pred)

#  Mostrar Metricas 

print(f"R^2 cdrag original: {r2_cdrag}, R^2 cdrag GPLearn: {r2_cdrag_gp} ")
print(f"MSE cdrag original: {mse_cdrag}, MSE cdrag GPLearn: {mse_cdrag_gp} ")
print(f"MAE cdrag original: {mae_cdrag}, MAE cdrag GPLearn: {mae_cdrag_gp} ")
print(f"MAPE cdrag original: {mape_cdrag*100}*100, MAPE cdrag GPLearn: {mape_cdrag_gp*100}*100 ")

C:\Users\lucas\AppData\Local\Temp\ipykernel_2232\2361799520.py:5: RuntimeWarning: overflow encountered in scalar power
  return S**(-0.056) + S*(6.450**Re)


ValueError: Input contains infinity or a value too large for dtype('float64').

### Ajuste automatico de constante V3

In [17]:
# A* x ^ B

def pot(x1, A, B):
    con1 = x1 > 0
    con2 = (B >= -1) & (B < 0)
    con3 = (A>=1) & (A<10)
    with np.errstate(divide='ignore', invalid='ignore'):
        result = np.where(con1 & con2 & con3, A*np.power(x1,B), 0)
    return result

pot_fn= make_function(function=pot, name='pot', arity=3)

In [18]:
# Modelo
model = SymbolicRegressor(population_size=7000,
                        generations=20, 
                        stopping_criteria=0.01,
                        const_range = (-1.0, 10.0),
                        p_crossover=0.7, 
                        p_subtree_mutation=0.1,
                        p_hoist_mutation=0.05, 
                        p_point_mutation=0.1,
                        max_samples=0.9, 
                        verbose=1,
                        parsimony_coefficient=1e-4, 
                        random_state=0,
                        function_set=['add', 'sub', 'mul', 'div',pot_fn],
                        metric = 'rmse', 
                        feature_names=['S', 'Re'])

model.fit(X_train, y_train)

# Obtener la expresión simbólica
expression = model._program
print("Expresión:", expression)

    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left


C:\Users\lucas\AppData\Local\Temp\ipykernel_3488\2416824652.py:8: RuntimeWarning: overflow encountered in power
  result = np.where(con1 & con2 & con3, A*np.power(x1,B), 0)
C:\Users\lucas\AppData\Local\Temp\ipykernel_3488\2416824652.py:8: RuntimeWarning: overflow encountered in multiply
  result = np.where(con1 & con2 & con3, A*np.power(x1,B), 0)


   0    60.02      4.93357e+19        8         0.192673          0.20065     38.60m
   1    43.77        8.862e+11        3         0.147749         0.152491     32.36m
   2    66.25      1.06883e+08        3         0.147998         0.150302     34.63m
   3    99.30      4.06181e+08        3         0.147223         0.157007     50.21m
   4    35.14      1.60696e+08        3         0.147126         0.157825     17.27m
   5    18.18       3.4603e+08       11          0.14405         0.144626      9.76m
   6     3.59      5.27281e+07       11         0.143378         0.150512      3.42m
   7     3.63      2.64035e+07       11          0.14344          0.14998      2.44m
   8     4.48           109347       11         0.142535         0.157556      2.22m
   9     8.12      3.24308e+07       11         0.142752         0.155775      2.41m
  10    10.97      5.36069e+07       11         0.142955         0.154093      2.68m
  11    11.20      1.01141e+12       11         0.142791         

In [19]:
# prediciendo con el conjunto de prueba

y_pred = model.predict(X_test)

# Calcular métricas
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)


print("MSE:", mse)
print("R^2:", r2)

MSE: 0.021153612653894996
R^2: 0.800101026692717


### Experimento: probando RMSLE para calcular el fitness

In [35]:
# A* x ^ B

def pot(x1, A, B):
    con1 = x1 > 0
    con2 = (B > -1) & (B < 0)
    con3 = (A>=1) & (A<10)
    with np.errstate(divide='ignore', invalid='ignore', over='ignore'):
        result = np.where(con1 & con2 & con3, A*np.power(x1,B), 0)
    return result

pot_fn= make_function(function=pot, name='pot', arity=3)

In [15]:
# definiendo 

def _rmsle(y, y_pred, w):
    """Calculate the rmsle."""
    log_true = np.log((np.maximum(0.001, y) + 1))
    log_pred = np.log((np.maximum(0.001, y_pred) + 1))
    
    # Calcular el error cuadrático medio logarítmico
    rmsle_value = (log_pred - log_true) ** 2
    
    return np.sqrt(np.average(rmsle_value, weights=w))

rmsle = make_fitness(function=_rmsle,
                    greater_is_better=False)

In [37]:
# Modelo
model = SymbolicRegressor(population_size=7000,
                        init_depth=(5, 9),
                        generations=20, 
                        stopping_criteria=0.01,
                        const_range = (-1.0, 10.0),
                        p_crossover=0.7, 
                        p_subtree_mutation=0.1,
                        p_hoist_mutation=0.05, 
                        p_point_mutation=0.1,
                        max_samples=0.9, 
                        verbose=1,
                        parsimony_coefficient=1e-3, 
                        random_state=0,
                        function_set=['add', 'sub', 'mul', 'div',pot_fn],
                        metric = rmsle, 
                        feature_names=['S', 'Re'])

model.fit(X_train, y_train)

# Obtener la expresión simbólica
expression = model._program
print("Expresión:", expression)

    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0   563.72          2.36735      110         0.100407        0.0977346    285.57m
   1   140.85         0.739198      104        0.0870931        0.0874299     70.84m
   2    77.52          0.70002       32        0.0646664        0.0660774     42.79m
   3    61.79          0.79886       99        0.0602612        0.0615921     31.66m
   4    20.49         0.990882       37        0.0480875          0.04882     12.88m
   5     6.99         0.829384       37        0.0478952        0.0504926      7.55m
   6     8.06         0.709249       24        0.0469346        0.0476777      6.75m
   7    17.53         0.745621       35        0.0465678           0.0457      9.87m
   8    20.97         0.678015       30        0.0465288        0.0471719  

### Experimento: probando RMSLE para calcular el fitness V2

In [4]:
# x ^ B

def pot(x1, B):
    con1 = x1 > 0
    con2 = (B > -1) & (B < 0)
    #con3 = (A>=1) & (A<10)
    with np.errstate(divide='ignore', invalid='ignore', over='ignore'):
        result = np.where(con1 & con2 , np.power(x1,B), 0)
    return result

pot_fn= make_function(function=pot, name='pot', arity=2)

def pot2(x1, A, B):
    con1 = x1 > 0
    con2 = (B > -1) & (B < 0)
    con3 = (A>=1) & (A<10)
    with np.errstate(divide='ignore', invalid='ignore', over='ignore'):
        result = np.where(con1 & con2 & con3 , A*np.power(x1,B), 0)
    return result

pot2_fn= make_function(function=pot2, name='pot2', arity=3)

In [25]:
# definiendo 

def _rmsle(y, y_pred, w):
    """Calculate the rmsle."""
    log_true = np.log((np.maximum(0.001, y) + 1))
    log_pred = np.log((np.maximum(0.001, y_pred) + 1))
    
    # Calcular el error cuadrático medio logarítmico
    rmsle_value = (log_pred - log_true) ** 2
    
    return np.sqrt(np.average(rmsle_value, weights=w))

rmsle = make_fitness(function=_rmsle,
                    greater_is_better=False)

In [9]:
# Modelo
model = SymbolicRegressor(population_size=7000,
                        init_depth=(5, 9),
                        generations=20, 
                        stopping_criteria=0.01,
                        const_range = (-1.0, 10.0),
                        p_crossover=0.9,
                        p_subtree_mutation=0.01, 
                        p_hoist_mutation=0.01, 
                        p_point_mutation=0.01, 
                        p_point_replace=0.05, 
                        max_samples=1.0,
                        verbose=1,
                        parsimony_coefficient=1e-4, 
                        random_state=0,
                        function_set=['add', 'sub', 'mul', 'div',pot_fn, pot2_fn],
                        metric = rmsle, 
                        feature_names=['S', 'Re'])

model.fit(X_train, y_train)

# Obtener la expresión simbólica
expression = model._program
print("Expresión:", expression)

    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0   517.99          1.39167      119        0.0570374              N/A    201.88m
   1   270.55         0.458137      123        0.0570374              N/A     93.21m
   2   327.15         0.295014       94        0.0515898              N/A    112.78m
   3   108.13         0.395088      113         0.045678              N/A     34.03m
   4    97.62         0.369287      117         0.045678              N/A     31.41m
   5    75.96         0.569397      143         0.045678              N/A     24.19m
   6    95.41         0.509861      110        0.0444863              N/A     28.50m
   7    73.24         0.581488      110        0.0444863              N/A     23.73m
   8    52.62          0.52264       52          0.04387              N/A  

In [10]:
# prediciendo con el conjunto de prueba

y_pred = model.predict(X_test)

# Calcular métricas
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)


print("MSE:", mse)
print("R^2:", r2)
print("MAE:", mae)
print("MAPE:",mape)

MSE: 0.012509492548400569
R^2: 0.8817868721558565
MAE: 0.08364321560876715
MAPE: 0.042665244354956916


In [11]:
# Modelo
model = SymbolicRegressor(population_size=7000,
                        init_depth=(5, 9),
                        generations=20, 
                        stopping_criteria=0.01,
                        const_range = (-1.0, 10.0),
                        p_crossover=0.9,
                        p_subtree_mutation=0.01, 
                        p_hoist_mutation=0.01, 
                        p_point_mutation=0.01, 
                        p_point_replace=0.05, 
                        max_samples=1.0,
                        verbose=1,
                        parsimony_coefficient=1e-3, 
                        random_state=0,
                        function_set=['add', 'sub', 'mul', 'div',pot_fn, pot2_fn],
                        metric = rmsle, 
                        feature_names=['S', 'Re'])

model.fit(X_train, y_train)

# Obtener la expresión simbólica
expression = model._program
print("Expresión:", expression)

    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0   517.99          1.39167      119        0.0570374              N/A    204.00m
   1   139.45          0.55073      123        0.0570374              N/A     50.53m
   2    89.61         0.513123       96        0.0449948              N/A     31.56m
   3    88.00         0.517237      106        0.0449184              N/A     27.99m
   4    56.89         0.599344       60        0.0424703              N/A     18.43m
   5    42.81          0.71871       59        0.0424703              N/A     14.29m
   6    31.06         0.682983       45        0.0422749              N/A     10.62m
   7    29.93         0.641195       45        0.0422552              N/A      8.75m
   8    32.35         0.567554       43        0.0420723              N/A  

In [12]:
# prediciendo con el conjunto de prueba

y_pred = model.predict(X_test)

# Calcular métricas
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)


print("MSE:", mse)
print("R^2:", r2)
print("MAE:", mae)
print("MAPE:",mape)

MSE: 0.018624350728263878
R^2: 0.8240022330933063
MAE: 0.10298233864038064
MAPE: 0.053830547896066866


In [14]:
def cdrag(S, Re):
    return S**(-0.6) + 5*Re**(-0.23)

def cdrag_gp(S, Re):
    return (S+1.061)/(S+0.202) + Re**(-0.273)

In [15]:
# Calcular cdrag_pred y cdrag_gp_pred en vectores separados
cdrag_pred = X_test.apply(lambda row: cdrag(row['K'], row['Rem']), axis=1).values
cdrag_gp_pred = X_test.apply(lambda row: cdrag_gp(row['K'], row['Rem']), axis=1).values

# Calcular R^2 y MSE para cdrag
r2_cdrag = r2_score(y_test['cdrag'], cdrag_pred)
mse_cdrag = mean_squared_error(y_test['cdrag'], cdrag_pred)

# Calcular MAE y MAPE para cdrag
mae_cdrag = mean_absolute_error(y_test['cdrag'], cdrag_pred)
mape_cdrag = mean_absolute_percentage_error(y_test['cdrag'], cdrag_pred)

# Calcular R^2 y MSE para cdrag_gp
r2_cdrag_gp = r2_score(y_test['cdrag'], cdrag_gp_pred)
mse_cdrag_gp = mean_squared_error(y_test['cdrag'], cdrag_gp_pred)

# Calcular MAE y MAPE para cdrag_gp
mae_cdrag_gp = mean_absolute_error(y_test['cdrag'], cdrag_gp_pred)
mape_cdrag_gp = mean_absolute_percentage_error(y_test['cdrag'], cdrag_gp_pred)

#  Mostrar Metricas 

print(f"R^2 cdrag original: {r2_cdrag}, R^2 cdrag GPLearn: {r2_cdrag_gp} ")
print(f"MSE cdrag original: {mse_cdrag}, MSE cdrag GPLearn: {mse_cdrag_gp} ")
print(f"MAE cdrag original: {mae_cdrag}, MAE cdrag GPLearn: {mae_cdrag_gp} ")
print(f"MAPE cdrag original: {mape_cdrag*100}*100, MAPE cdrag GPLearn: {mape_cdrag_gp*100}*100 ")

R^2 cdrag original: 0.8904926780136748, R^2 cdrag GPLearn: 0.8240567720947451 
MSE cdrag original: 0.011588230963563853, MSE cdrag GPLearn: 0.018618579328381825 
MAE cdrag original: 0.07580331737181549, MAE cdrag GPLearn: 0.10297576458038689 
MAPE cdrag original: 3.7448292462961463*100, MAPE cdrag GPLearn: 5.383526642536919*100 


In [16]:
model = SymbolicRegressor(population_size=7000,
                        init_depth=(5, 9),
                        generations=20, 
                        stopping_criteria=0.01,
                        const_range = (-1.0, 10.0),
                        p_crossover=0.9,
                        p_subtree_mutation=0.01, 
                        p_hoist_mutation=0.01, 
                        p_point_mutation=0.01, 
                        p_point_replace=0.05, 
                        max_samples=1.0,
                        verbose=1,
                        parsimony_coefficient=1e-2, 
                        random_state=0,
                        function_set=['add', 'sub', 'mul', 'div',pot_fn, pot2_fn],
                        metric = rmsle, 
                        feature_names=['S', 'Re'])

model.fit(X_train, y_train)

# Obtener la expresión simbólica
expression = model._program
print("Expresión:", expression)

    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0   517.99          1.39167      119        0.0570374              N/A    186.70m
   1    56.90         0.945585      119        0.0570374              N/A     21.03m
   2    20.89         0.882782       38        0.0636891              N/A      9.26m
   3     5.63         0.715348        9        0.0464463              N/A      3.68m
   4     2.42         0.335649        9        0.0464463              N/A      2.74m
   5     1.52          0.14475        5        0.0468234              N/A      2.28m
   6     3.19          0.19225        5        0.0468234              N/A      2.46m
   7     3.07         0.202594        3        0.0540941              N/A      2.13m
   8     3.10          0.19561        3        0.0540941              N/A  

### Experimento: probando MALE para calcular el fitness

In [11]:
# definiendo 

def _male(y, y_pred, w):
    """Calculate the male."""
    log_true = np.log((np.maximum(0.001, y) + 1))
    log_pred = np.log((np.maximum(0.001, y_pred) + 1))
    
    # Calcular el error cuadrático medio logarítmico
    male_value = np.abs((log_pred - log_true))
    
    return np.sqrt(np.average(male_value, weights=w))

male = make_fitness(function=_male,
                    greater_is_better=False)

In [12]:
# Modelo
model = SymbolicRegressor(population_size=7000,
                        init_depth=(5, 9),
                        generations=20, 
                        stopping_criteria=0.01,
                        const_range = (-1.0, 10.0),
                        p_crossover=0.9,
                        p_subtree_mutation=0.01, 
                        p_hoist_mutation=0.01, 
                        p_point_mutation=0.01, 
                        p_point_replace=0.05, 
                        max_samples=1.0,
                        verbose=1,
                        parsimony_coefficient=1e-4, 
                        random_state=0,
                        function_set=['add', 'sub', 'mul', 'div',pot_fn, pot2_fn],
                        metric = male, 
                        feature_names=['S', 'Re'])

model.fit(X_train, y_train)

# Obtener la expresión simbólica
expression = model._program
print("Expresión:", expression)

    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0   517.99          1.05587      119         0.215437              N/A    192.56m
   1   259.58         0.613986      123         0.215437              N/A    108.91m
   2   346.57         0.405068      123         0.214324              N/A    146.62m
   3   115.96         0.417321       84         0.190398              N/A     78.43m
   4    90.38         0.439365       94         0.175326              N/A     73.69m
   5    64.18         0.528701       97         0.175326              N/A     49.56m
   6    73.09          0.51716       98         0.173164              N/A     37.03m
   7    89.80         0.472591       99         0.166231              N/A     22.36m
   8    85.97         0.445841       95         0.166231              N/A  

### Experimento: escalar las variables

    Conclusión: no funcionó. No se considera el Reynolds, o solía aproximar la función a una ecuación lineal.

In [4]:
# definiendo conjunto de train
df_cdrag_train = pd.concat([df_cdrag_25, df_cdrag_74], ignore_index=True)

# separando entre x e y
y_train = df_cdrag_train.drop(columns=['Current','K','Flujo','t_viento','Diametro','col_fluido','col_celda','n_fluido','n_celda','Rem','colIndex'])
X_train = df_cdrag_train.drop(columns=['Current','Flujo','t_viento','Diametro','col_fluido','col_celda','n_fluido',
                                    'n_celda','colIndex','cdrag'])


# definiendo conjunto de test

y_test = df_cdrag_53.drop(columns=['Current','K','Flujo','t_viento','Diametro','col_fluido','col_celda','n_fluido','n_celda','Rem','colIndex'])
X_test = df_cdrag_53.drop(columns=['Current','Flujo','t_viento','Diametro','col_fluido','col_celda','n_fluido',
                                    'n_celda','colIndex','cdrag'])

In [5]:
scaler = MinMaxScaler()

X_train_scaled = scaler.fit_transform(X_train)

y_train_scaled = scaler.fit_transform(y_train)


In [6]:
# X1 ^ B

def pot(x1,B):
    con1 = x1 > 0
    con2 = (B > -1) & (B < 0)
    #con3 = (A>=1) & (A<10)
    with np.errstate(divide='ignore', invalid='ignore', over='ignore'):
        result = np.where(con1 & con2 ,np.power(x1,B), 0)
    return result

pot_fn= make_function(function=pot, name='pot', arity=2)

# A* X1 ^B
def pot2(x1,A, B):
    con1 = x1 > 0
    con2 = (B > -1) & (B < 0)
    con3 = (A>=1) & (A<10)
    with np.errstate(divide='ignore', invalid='ignore', over='ignore'):
        result = np.where(con1 & con2 & con3 ,A*np.power(x1,B), 0)
    return result

pot2_fn= make_function(function=pot2, name='pot2', arity=3)

In [8]:
# Modelo
model = SymbolicRegressor(population_size=7000,
                        init_depth=(5, 9),
                        generations=20, 
                        stopping_criteria=0.01,
                        const_range = (-1.0, 10.0),
                        p_crossover=0.9,
                        p_subtree_mutation=0.01, 
                        p_hoist_mutation=0.01, 
                        p_point_mutation=0.01, 
                        p_point_replace=0.05, 
                        max_samples=1.0,
                        verbose=1,
                        parsimony_coefficient=1e-3, 
                        random_state=0,
                        function_set=['add', 'sub', 'mul', 'div',pot_fn, pot2_fn],
                        metric = rmsle, 
                        feature_names=['S', 'Re'])

model.fit(X_train_scaled,y_train_scaled)

# Obtener la expresión simbólica
expression = model._program
print("Expresión:", expression)

d:\env\lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0   517.99         0.400563     1691        0.0620841              N/A    189.10m
   1    53.81         0.305262      213        0.0636314              N/A     17.21m
   2    35.65         0.294892       31        0.0572523              N/A     12.14m
   3    30.07         0.243388       31        0.0572523              N/A      8.82m
   4    15.23         0.272991       34         0.052681              N/A      4.92m
   5    14.56         0.257746       13        0.0395187              N/A      5.54m
   6    22.65         0.197383       13        0.0395187              N/A      6.90m
   7    17.39         0.211856       13        0.0393614              N/A      5.20m
   8    12.46         0.214674       13        0.0393614              N/A  

In [11]:
# prediciendo con el conjunto de prueba

X_test_scaled = scaler.fit_transform(X_test)
y_test_scaled = scaler.fit_transform(y_test)

y_pred = model.predict(X_test_scaled)

# Calcular métricas
mse = mean_squared_error(y_test_scaled, y_pred)
r2 = r2_score(y_test_scaled, y_pred)
mae = mean_absolute_error(y_test_scaled, y_pred)
mape = mean_absolute_percentage_error(y_test_scaled, y_pred)


print("MSE:", mse)
print("R^2:", r2)
print("MAE:", mae)
print("MAPE:",mape)

MSE: 0.0035734637729696386
R^2: 0.8677315986042082
MAE: 0.044905556187363874
MAPE: 24885354781.94131


### Ajuste automatico de constante Vn

In [9]:
# X1 ^ B

def pot(x1,B):
    con1 = x1 > 0
    con2 = (B > -1) & (B < 0)
    #con3 = (A>=1) & (A<10)
    with np.errstate(divide='ignore', invalid='ignore', over='ignore'):
        result = np.where(con1 & con2 ,np.power(x1,B), 0)
    return result

pot_fn= make_function(function=pot, name='pot', arity=2)

# 5* X1 ^B
def pot2(x1, B):
    con1 = x1 > 0
    con2 = (B > -1) & (B < 0)
    #con3 = (A>=1) & (A<10)
    with np.errstate(divide='ignore', invalid='ignore', over='ignore'):
        result = np.where(con1 & con2 ,5*np.power(x1,B), 0)
    return result

pot2_fn= make_function(function=pot2, name='pot2', arity=2)

In [13]:
# Modelo
model = SymbolicRegressor(population_size=5000,
                        generations=20, 
                        stopping_criteria=0.01,
                        p_crossover=0.7, 
                        p_subtree_mutation=0.1,
                        p_hoist_mutation=0.05, 
                        p_point_mutation=0.1,
                        max_samples=0.9, 
                        verbose=1,
                        parsimony_coefficient=0.001, 
                        random_state=0,
                        function_set=['add', 'sub', 'mul', 'div',pot_fn, pot2_fn],
                        metric = rmsle, 
                        feature_names=['S', 'Re'])

model.fit(X_train, y_train)

# Obtener la expresión simbólica
expression = model._program
print("Expresión:", expression)

    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0    40.13          1.63734       63        0.0609554         0.060432     29.69m
   1    27.90          0.88399        5        0.0597663        0.0592614     21.08m
   2    35.11         0.825523      109        0.0576371        0.0569709     21.45m
   3    33.09         0.723208        5        0.0455831        0.0473043     22.93m
   4    25.66          0.70909        5        0.0457082        0.0462047     17.72m
   5    12.40         0.608055        5        0.0455714        0.0474058      7.14m
   6     6.06         0.471864        5        0.0454279        0.0486292      3.74m
   7     5.49         0.409861        5        0.0453854        0.0489855      3.84m
   8     5.26         0.427488        5        0.0453314        0.0494328  

In [14]:
# prediciendo con el conjunto de prueba

y_pred = model.predict(X_test)

# Calcular métricas
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)


print("MSE:", mse)
print("R^2:", r2)
print("MAE:", mae)
print("MAPE:",mape)

MSE: 0.018859381225995866
R^2: 0.8217812245137683
MAE: 0.10151616114359971
MAPE: 0.051538457694427275


In [15]:
def cdrag(S, Re):
    return S**(-0.6) + 5*Re**(-0.23)

def cdrag_gp(S, Re):
    return S**(-0.656) + Re**(-0.031)

In [16]:
# Calcular cdrag_pred y cdrag_gp_pred en vectores separados
cdrag_pred = X_test.apply(lambda row: cdrag(row['K'], row['Rem']), axis=1).values
cdrag_gp_pred = X_test.apply(lambda row: cdrag_gp(row['K'], row['Rem']), axis=1).values

# Calcular R^2 y MSE para cdrag
r2_cdrag = r2_score(y_test['cdrag'], cdrag_pred)
mse_cdrag = mean_squared_error(y_test['cdrag'], cdrag_pred)

# Calcular MAE y MAPE para cdrag
mae_cdrag = mean_absolute_error(y_test['cdrag'], cdrag_pred)
mape_cdrag = mean_absolute_percentage_error(y_test['cdrag'], cdrag_pred)

# Calcular R^2 y MSE para cdrag_gp
r2_cdrag_gp = r2_score(y_test['cdrag'], cdrag_gp_pred)
mse_cdrag_gp = mean_squared_error(y_test['cdrag'], cdrag_gp_pred)

# Calcular MAE y MAPE para cdrag_gp
mae_cdrag_gp = mean_absolute_error(y_test['cdrag'], cdrag_gp_pred)
mape_cdrag_gp = mean_absolute_percentage_error(y_test['cdrag'], cdrag_gp_pred)

#  Mostrar Metricas 

print(f"R^2 cdrag original: {r2_cdrag}, R^2 cdrag GPLearn: {r2_cdrag_gp} ")
print(f"MSE cdrag original: {mse_cdrag}, MSE cdrag GPLearn: {mse_cdrag_gp} ")
print(f"MAE cdrag original: {mae_cdrag}, MAE cdrag GPLearn: {mae_cdrag_gp} ")
print(f"MAPE cdrag original: {mape_cdrag*100}*100, MAPE cdrag GPLearn: {mape_cdrag_gp*100}*100 ")

R^2 cdrag original: 0.8904926780136748, R^2 cdrag GPLearn: 0.8231231569051083 
MSE cdrag original: 0.011588230963563853, MSE cdrag GPLearn: 0.018717375904285256 
MAE cdrag original: 0.07580331737181549, MAE cdrag GPLearn: 0.10101354711145066 
MAPE cdrag original: 3.7448292462961463*100, MAPE cdrag GPLearn: 5.135779418892614*100 


In [18]:
# Modelo
model = SymbolicRegressor(population_size=5000,
                        generations=20, 
                        const_range=(-1,8),
                        stopping_criteria=0.01,
                        p_crossover=0.7, 
                        p_subtree_mutation=0.1,
                        p_hoist_mutation=0.05, 
                        p_point_mutation=0.1,
                        max_samples=0.9, 
                        verbose=1,
                        parsimony_coefficient=0.001, 
                        random_state=0,
                        function_set=['add', 'sub', 'mul', 'div',pot_fn, pot2_fn],
                        #metric = rmsle, 
                        feature_names=['S', 'Re'])

model.fit(X_train, y_train)

# Obtener la expresión simbólica
expression = model._program
print("Expresión:", expression)

    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0    40.13      1.26092e+11        7          0.22805         0.225836     24.12m
   1    25.30           124020        7         0.227215         0.233356     14.42m
   2    32.01          52009.5        5         0.137923         0.139003     17.07m
   3    13.79          45231.2        7         0.114091         0.113102      7.64m
   4     6.84      1.99642e+07       13         0.106464         0.105984      4.10m
   5     6.38          27784.4        9         0.103681         0.107175      3.16m
   6     7.80      1.21515e+07        9         0.103775         0.106329      4.14m
   7     4.47          31609.9        9         0.103531          0.10852      2.39m
   8     4.11      6.78816e+07        9         0.103127         0.105615  

In [19]:
# prediciendo con el conjunto de prueba

y_pred = model.predict(X_test)

# Calcular métricas
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)


print("MSE:", mse)
print("R^2:", r2)
print("MAE:", mae)
print("MAPE:",mape)

MSE: 0.020234249519583917
R^2: 0.8087888924323552
MAE: 0.09877020005507078
MAPE: 0.05046559180367911


In [6]:
def cdrag(S, Re):
    return S**(-0.6) + 5*Re**(-0.23)

def cdrag_gp(S, Re):
    return (Re/S)**(-0.275) + ((S + 0.673324)/S)

In [7]:
# Calcular cdrag_pred y cdrag_gp_pred en vectores separados
cdrag_pred = X_test.apply(lambda row: cdrag(row['K'], row['Rem']), axis=1).values
cdrag_gp_pred = X_test.apply(lambda row: cdrag_gp(row['K'], row['Rem']), axis=1).values

# Calcular R^2 y MSE para cdrag
r2_cdrag = r2_score(y_test['cdrag'], cdrag_pred)
mse_cdrag = mean_squared_error(y_test['cdrag'], cdrag_pred)

# Calcular MAE y MAPE para cdrag
mae_cdrag = mean_absolute_error(y_test['cdrag'], cdrag_pred)
mape_cdrag = mean_absolute_percentage_error(y_test['cdrag'], cdrag_pred)

# Calcular R^2 y MSE para cdrag_gp
r2_cdrag_gp = r2_score(y_test['cdrag'], cdrag_gp_pred)
mse_cdrag_gp = mean_squared_error(y_test['cdrag'], cdrag_gp_pred)

# Calcular MAE y MAPE para cdrag_gp
mae_cdrag_gp = mean_absolute_error(y_test['cdrag'], cdrag_gp_pred)
mape_cdrag_gp = mean_absolute_percentage_error(y_test['cdrag'], cdrag_gp_pred)

#  Mostrar Metricas 

print(f"R^2 cdrag original: {r2_cdrag}, R^2 cdrag GPLearn: {r2_cdrag_gp} ")
print(f"MSE cdrag original: {mse_cdrag}, MSE cdrag GPLearn: {mse_cdrag_gp} ")
print(f"MAE cdrag original: {mae_cdrag}, MAE cdrag GPLearn: {mae_cdrag_gp} ")
print(f"MAPE cdrag original: {mape_cdrag*100} %, MAPE cdrag GPLearn: {mape_cdrag_gp*100} %")

R^2 cdrag original: 0.8904926780136748, R^2 cdrag GPLearn: 0.8087713914878053 
MSE cdrag original: 0.011588230963563853, MSE cdrag GPLearn: 0.020236101496089648 
MAE cdrag original: 0.07580331737181549, MAE cdrag GPLearn: 0.09876316341817992 
MAPE cdrag original: 3.7448292462961463*100, MAPE cdrag GPLearn: 5.0466013800943825*100 


### Experimentos: tomando como base el experimento  que resultó más parecido

In [48]:
# definiendo 

def _rmsle(y, y_pred, w):
    """Calculate the rmsle."""
    log_true = np.log((np.maximum(0.001, y) + 1))
    log_pred = np.log((np.maximum(0.001, y_pred) + 1))
    
    # Calcular el error cuadrático medio logarítmico
    rmsle_value = (log_pred - log_true) ** 2
    
    return np.sqrt(np.average(rmsle_value, weights=w))

rmsle = make_fitness(function=_rmsle,
                    greater_is_better=False)

In [49]:
# x ^ B

def pot(x1, B):
    con1 = x1 > 0
    con2 = (B > -1) & (B < 0)
    #con3 = (A>=1) & (A<10)
    with np.errstate(divide='ignore', invalid='ignore', over='ignore'):
        result = np.where(con1 & con2 , np.power(x1,B), 0)
    return result

pot_fn= make_function(function=pot, name='pot', arity=2)

def pot2(x1, A, B):
    con1 = x1 > 0
    con2 = (B > -1) & (B < 0)
    con3 = (A>1) & (A<8)
    with np.errstate(divide='ignore', invalid='ignore', over='ignore'):
        result = np.where(con1 & con2 & con3 , A*np.power(x1,B), 0)
    return result

pot2_fn= make_function(function=pot2, name='pot2', arity=3)

In [8]:
# Modelo
model = SymbolicRegressor(population_size=5000,
                        generations=20, 
                        stopping_criteria=0.01,
                        p_crossover=0.7, 
                        p_subtree_mutation=0.1,
                        p_hoist_mutation=0.05, 
                        p_point_mutation=0.1,
                        max_samples=0.9, 
                        verbose=1,
                        parsimony_coefficient=0.0001,
                        const_range= (-1.0,8.0),
                        random_state=0,
                        function_set=['add', 'sub', 'mul', 'div',pot_fn, pot2_fn],
                        metric = rmsle, 
                        feature_names=['S', 'Re'])

model.fit(X_train, y_train)

# Obtener la expresión simbólica
expression = model._program
print("Expresión:", expression)

# prediciendo con el conjunto de prueba

y_pred = model.predict(X_test)

# Calcular métricas
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)


print("MSE:", mse)
print("R^2:", r2)
print("MAE:", mae)
print("MAPE:",mape)

    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0    56.37          1.75501       50        0.0923517        0.0913985     34.16m
   1    37.95         0.850732        9        0.0694237        0.0699577     18.85m
   2    68.50         0.647001       22        0.0489846        0.0501482     28.26m
   3    39.44          0.50415       22        0.0489074        0.0508216     24.08m
   4    28.86         0.625064       19         0.048793        0.0565932     15.31m
   5    30.13         0.508955       19        0.0473034        0.0456795     14.14m
   6    11.52         0.651198       19         0.046933        0.0489973      4.85m
   7     4.11         0.406052       19         0.046745         0.050589      2.96m
   8     7.59         0.488108       17        0.0467047        0.0509229  

In [50]:
# Modelo
model = SymbolicRegressor(population_size=5000,
                        generations=20, 
                        stopping_criteria=0.01,
                        p_crossover=0.7, 
                        p_subtree_mutation=0.1,
                        p_hoist_mutation=0.05, 
                        p_point_mutation=0.1,
                        max_samples=0.9, 
                        verbose=1,
                        parsimony_coefficient=0.0005,
                        const_range= (-1.0,8.0),
                        random_state=0,
                        function_set=['add', 'sub', 'mul', 'div',pot_fn, pot2_fn],
                        metric = rmsle, 
                        feature_names=['S', 'Re'])

model.fit(X_train, y_train)

# Obtener la expresión simbólica
expression = model._program
print("Expresión:", expression)

# prediciendo con el conjunto de prueba

y_pred = model.predict(X_test)

# Calcular métricas
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)


print("MSE:", mse)
print("R^2:", r2)
print("MAE:", mae)
print("MAPE:",mape)

    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0    56.37          1.75501       50        0.0923517        0.0913985     33.22m
   1    36.54         0.860319        9        0.0694237        0.0699577     23.15m
   2    56.43         0.698339       22        0.0489846        0.0501482     30.29m
   3    23.44          0.85118       22        0.0489074        0.0508216     14.68m
   4    16.20         0.983833       11        0.0478553         0.046934      9.60m
   5    23.37          0.59883       19        0.0473034        0.0456795     10.50m
   6     8.71         0.569801       19         0.046933        0.0489973      3.92m
   7     3.40         0.375458        3        0.0491972        0.0529096      2.81m
   8     3.47         0.397946        3        0.0491611        0.0532104  

In [9]:
# Modelo
model = SymbolicRegressor(population_size=5000,
                        generations=20, 
                        stopping_criteria=0.01,
                        p_crossover=0.7, 
                        p_subtree_mutation=0.1,
                        p_hoist_mutation=0.05, 
                        p_point_mutation=0.1,
                        max_samples=0.9, 
                        verbose=1,
                        parsimony_coefficient=0.0001,
                        const_range= (-1.0,8.0),
                        random_state=0,
                        function_set=['add', 'sub', 'mul', 'div',pot_fn, pot2_fn],
                        #metric = rmsle, 
                        feature_names=['S', 'Re'])

model.fit(X_train, y_train)

# Obtener la expresión simbólica
expression = model._program
print("Expresión:", expression)

# prediciendo con el conjunto de prueba

y_pred = model.predict(X_test)

# Calcular métricas
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)


print("MSE:", mse)
print("R^2:", r2)
print("MAE:", mae)
print("MAPE:",mape)

    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0    56.37       1.3351e+11       50         0.242531         0.239706     30.15m
   1    37.97      2.60658e+08      178         0.143973         0.142393     19.77m
   2    70.71           106356       19         0.109815         0.110065     31.71m
   3    38.63      1.22593e+06       19         0.109389         0.113897     16.88m
   4    97.01      4.15771e+07       19         0.108969         0.117671     32.36m
   5    56.54       1.6338e+06       17         0.107266         0.114041     17.40m
   6    18.33      3.07708e+07       11         0.108225         0.111264      5.74m
   7    15.08          21738.1       17         0.107204           0.1146      3.55m
   8    14.13          13224.5       17         0.107267         0.114033  

In [10]:
# Modelo
model = SymbolicRegressor(population_size=5000,
                        generations=20, 
                        stopping_criteria=0.01,
                        p_crossover=0.7, 
                        p_subtree_mutation=0.1,
                        p_hoist_mutation=0.05, 
                        p_point_mutation=0.1,
                        max_samples=0.9, 
                        verbose=1,
                        parsimony_coefficient=0.001,
                        const_range= (-1.0,8.0),
                        random_state=0,
                        function_set=['add', 'sub', 'mul', 'div',pot_fn, pot2_fn],
                        #metric = rmsle, 
                        feature_names=['S', 'Re'])

model.fit(X_train, y_train)

# Obtener la expresión simbólica
expression = model._program
print("Expresión:", expression)

# prediciendo con el conjunto de prueba

y_pred = model.predict(X_test)

# Calcular métricas
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)


print("MSE:", mse)
print("R^2:", r2)
print("MAE:", mae)
print("MAPE:",mape*100)

    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0    56.37       1.3351e+11       50         0.242531         0.239706     32.28m
   1    34.83      2.60649e+08      178         0.143973         0.142393     26.39m
   2    58.97           103284       19         0.109815         0.110065     29.66m
   3    24.40      5.99191e+07       19         0.109389         0.113897     12.38m
   4    10.31      9.35729e+07       11          0.10883         0.105817      5.93m
   5    17.86      1.85614e+07       17         0.107483         0.112088      8.62m
   6     5.20      3.06696e+07       17         0.107869         0.108614      2.64m
   7     3.39            40179        3         0.113536         0.121925      1.95m
   8     3.47           140653        3         0.113514         0.122122  

In [45]:
# Modelo
model = SymbolicRegressor(population_size=5000,
                        generations=20, 
                        stopping_criteria=0.01,
                        p_crossover=0.6, 
                        p_subtree_mutation=0.1,
                        p_hoist_mutation=0.05, 
                        p_point_mutation=0.1,
                        max_samples=1.0, 
                        verbose=1,
                        parsimony_coefficient=0.001,
                        const_range= (-1.0,8.0),
                        random_state=0,
                        function_set=['add', 'sub', 'mul', 'div',pot_fn, pot2_fn],
                        metric = rmsle, 
                        feature_names=['S', 'Re'])

model.fit(X_train, y_train)

# Obtener la expresión simbólica
expression = model._program
print("Expresión:", expression)

# prediciendo con el conjunto de prueba

y_pred = model.predict(X_test)

# Calcular métricas
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)


print("MSE:", mse)
print("R^2:", r2)
print("MAE:", mae)
print("MAPE:",mape)

    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0    56.37          1.75503       50        0.0922568              N/A     16.73m
   1    33.74         0.828189       32         0.076318              N/A     10.24m
   2    44.46         0.688555       48        0.0495809              N/A     10.85m
   3    18.03         0.792213       48        0.0495809              N/A      6.00m
   4    14.07         0.861008       53        0.0495809              N/A      4.27m
   5    17.44         0.595667        3        0.0495809              N/A      5.39m
   6     8.14         0.556969        7        0.0495761              N/A      2.56m
   7     3.52         0.331636        3        0.0495809              N/A      1.50m
   8     3.47         0.381763        3        0.0495809              N/A  

In [46]:
# Modelo
model = SymbolicRegressor(population_size=5000,
                        generations=20, 
                        stopping_criteria=0.01,
                        p_crossover=0.6, 
                        p_subtree_mutation=0.05,
                        p_hoist_mutation=0.05, 
                        p_point_mutation=0.05,
                        max_samples=1.0, 
                        verbose=1,
                        parsimony_coefficient=0.001,
                        const_range= (-1.0,8.0),
                        random_state=0,
                        function_set=['add', 'sub', 'mul', 'div',pot_fn, pot2_fn],
                        metric = rmsle, 
                        feature_names=['S', 'Re'])

model.fit(X_train, y_train)

# Obtener la expresión simbólica
expression = model._program
print("Expresión:", expression)

# prediciendo con el conjunto de prueba

y_pred = model.predict(X_test)

# Calcular métricas
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)


print("MSE:", mse)
print("R^2:", r2)
print("MAE:", mae)
print("MAPE:",mape)

    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0    56.37          1.75503       50        0.0922568              N/A     25.85m
   1    33.73         0.784317       32         0.076318              N/A     18.51m
   2    44.45         0.646463       48        0.0495809              N/A     20.51m
   3    17.16         0.720742       48        0.0495809              N/A      5.49m
   4    14.29         0.658432       53        0.0495809              N/A      8.62m
   5    12.40         0.531294       40        0.0495809              N/A      5.73m
   6     4.01         0.332885        3        0.0495809              N/A      1.70m
   7     3.20         0.226432        3        0.0495809              N/A      1.29m
   8     3.25         0.251667        3        0.0495809              N/A  

In [47]:
# Modelo
model = SymbolicRegressor(population_size=5000,
                        generations=20, 
                        stopping_criteria=0.01,
                        p_crossover=0.6, 
                        p_subtree_mutation=0.05,
                        p_hoist_mutation=0.05, 
                        p_point_mutation=0.05,
                        max_samples=1.0, 
                        verbose=1,
                        parsimony_coefficient=0.001,
                        const_range= (-1.0,8.0),
                        random_state=0,
                        function_set=['add', 'sub', 'mul', 'div',pot_fn, pot2_fn],
                        #metric = rmsle, 
                        feature_names=['S', 'Re'])

model.fit(X_train, y_train)

# Obtener la expresión simbólica
expression = model._program
print("Expresión:", expression)

# prediciendo con el conjunto de prueba

y_pred = model.predict(X_test)

# Calcular métricas
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)


print("MSE:", mse)
print("R^2:", r2)
print("MAE:", mae)
print("MAPE:",mape)

    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0    56.37      1.33638e+11       37         0.242223              N/A     16.71m
   1    34.73      1.20269e+08       32         0.181036              N/A      9.73m
   2    59.11           103177       48         0.114375              N/A     14.06m
   3    23.13          79591.4       47         0.114019              N/A      6.48m
   4    23.41           391714       55         0.114019              N/A      6.11m
   5    30.24          56241.9       49         0.114019              N/A      8.06m
   6     5.90          50147.4       24         0.114019              N/A      1.93m
   7     3.23      1.83907e+07        3         0.114375              N/A      1.12m
   8     3.25          67917.4        3         0.114375              N/A  

### Experimento: con solo una funcion potencia

Se demora demasiado en converger. 

In [4]:

def _rmsle(y, y_pred, w):
    """Calculate the rmsle."""
    log_true = np.log((np.maximum(0.001, y) + 1))
    log_pred = np.log((np.maximum(0.001, y_pred) + 1))
    
    # Calcular el error cuadrático medio logarítmico
    rmsle_value = (log_pred - log_true) ** 2
    
    return np.sqrt(np.average(rmsle_value, weights=w))

rmsle = make_fitness(function=_rmsle,
                    greater_is_better=False)

### Nuevas combinaciones

In [11]:
def pot(x1,B):
    con1 = x1 > 0
    con2 = (B > -1) & (B < 0)
    #con3 = (A>=1) & (A<8)
    with np.errstate(divide='ignore', invalid='ignore', over='ignore'):
        result = np.where(con1 & con2  ,np.power(x1,B), 0)
    return result

pot_fn= make_function(function=pot, name='pot', arity=2)

def pot2(x1,B):
    con1 = x1 > 0
    con2 = (B > -1) & (B < 0)
    #con3 = (A>=1) & (A<8)
    with np.errstate(divide='ignore', invalid='ignore', over='ignore'):
        result = np.where(con1 & con2  ,np.dot(4,np.power(x1,B)), 0)
    return result

pot2_fn= make_function(function=pot2, name='pot2', arity=2)

In [14]:
# Modelo
model = SymbolicRegressor(population_size=7000,
                        init_depth=(5, 9),
                        generations=20, 
                        stopping_criteria=0.01,
                        const_range = (-1.0, 10.0),
                        p_crossover=0.9,
                        p_subtree_mutation=0.01, 
                        p_hoist_mutation=0.01, 
                        p_point_mutation=0.01, 
                        p_point_replace=0.05, 
                        max_samples=1.0,
                        verbose=1,
                        parsimony_coefficient=1e-3, 
                        random_state=0,
                        function_set=['add', 'sub', 'mul', 'div',pot_fn,pot2_fn],
                        #metric = rmsle, 
                        feature_names=['S', 'Re'])

model.fit(X_train, y_train)

# Obtener la expresión simbólica
expression = model._program
print("Expresión:", expression)

    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0   291.55       1.7173e+21      111         0.190911              N/A    117.20m
   1   137.68      2.62747e+07      111         0.190911              N/A     50.73m
   2   133.02      8.74897e+10       89         0.190911              N/A     46.02m
   3    69.74      6.47508e+11      115         0.150901              N/A     23.05m
   4    59.87           116388       31         0.109058              N/A     19.23m
   5    42.46      1.27483e+08       31         0.108969              N/A     13.00m
   6    35.42      9.08531e+10       27         0.107343              N/A     11.08m
   7    41.39      1.01484e+08       31         0.107343              N/A     11.11m
   8    36.43           294194       33         0.107325              N/A  

In [15]:
# prediciendo con el conjunto de prueba

y_pred = model.predict(X_test)

# Calcular métricas
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)


print("MSE:", mse)
print("R^2:", r2)
print("MAE:", mae)
print("MAPE:",mape)

MSE: 0.02445910730007164
R^2: 0.7688645189219384
MAE: 0.11674923666488987
MAPE: 0.05985176074507344


In [16]:
# Modelo
model = SymbolicRegressor(population_size=7000,
                        init_depth=(5, 9),
                        generations=20, 
                        stopping_criteria=0.01,
                        const_range = (-1.0, 10.0),
                        p_crossover=0.9,
                        p_subtree_mutation=0.01, 
                        p_hoist_mutation=0.01, 
                        p_point_mutation=0.01, 
                        p_point_replace=0.05, 
                        max_samples=1.0,
                        verbose=1,
                        parsimony_coefficient=1e-3, 
                        random_state=0,
                        function_set=['add', 'sub', 'mul', 'div',pot_fn],
                        #metric = rmsle, 
                        feature_names=['S', 'Re'])

model.fit(X_train, y_train)

# Obtener la expresión simbólica
expression = model._program
print("Expresión:", expression)

    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0   281.38      1.88996e+24       67         0.237109              N/A     78.93m
   1   140.82      8.75988e+17        3         0.123755              N/A     36.92m
   2   112.23           371630        3         0.123755              N/A     28.31m
   3    44.25           153234        3         0.123755              N/A     13.73m
   4    19.86          68494.6        3         0.123755              N/A      6.45m
   5    37.69      1.41324e+07       63         0.108781              N/A      8.69m
   6    22.71          76378.2       63         0.108781              N/A      5.49m
   7     4.31          50198.3       37         0.112379              N/A      1.93m
   8     3.15       1.0296e+08        3         0.123755              N/A  

In [17]:
# Modelo
model = SymbolicRegressor(population_size=7000,
                        init_depth=(5, 9),
                        generations=20, 
                        stopping_criteria=0.01,
                        const_range = (-1.0, 10.0),
                        p_crossover=0.9,
                        p_subtree_mutation=0.01, 
                        p_hoist_mutation=0.01, 
                        p_point_mutation=0.01, 
                        p_point_replace=0.05, 
                        max_samples=1.0,
                        verbose=1,
                        parsimony_coefficient=1e-3, 
                        random_state=0,
                        function_set=['add', 'sub', 'mul', 'div',pot_fn],
                        metric = rmsle, 
                        feature_names=['S', 'Re'])

model.fit(X_train, y_train)

# Obtener la expresión simbólica
expression = model._program
print("Expresión:", expression)

    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0   281.38          2.43121     1023         0.100842              N/A     88.27m
   1   113.95         0.670959        3        0.0530797              N/A     30.64m
   2    74.54         0.595175        3        0.0530797              N/A     20.35m
   3    39.41         0.591902        3        0.0530797              N/A     13.06m
   4    14.66         0.902904        3        0.0530797              N/A      5.62m
   5    16.26          1.27397       37        0.0468881              N/A      6.06m
   6     8.68         0.790204       37        0.0468881              N/A      3.33m
   7     3.20         0.218879        7        0.0483221              N/A      2.08m
   8     4.23         0.267706        9        0.0473653              N/A  

In [18]:
# Modelo
model = SymbolicRegressor(population_size=7000,
                        init_depth=(5, 9),
                        generations=20, 
                        stopping_criteria=0.01,
                        const_range = (-1.0, 10.0),
                        p_crossover=0.9,
                        p_subtree_mutation=0.01, 
                        p_hoist_mutation=0.01, 
                        p_point_mutation=0.01, 
                        p_point_replace=0.05, 
                        max_samples=1.0,
                        verbose=1,
                        parsimony_coefficient=1e-3, 
                        random_state=0,
                        function_set=['add', 'sub', 'mul', 'div',pot_fn,pot2_fn],
                        metric = rmsle, 
                        feature_names=['S', 'Re'])

model.fit(X_train, y_train)

# Obtener la expresión simbólica
expression = model._program
print("Expresión:", expression)

    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0   291.55          1.35346      111        0.0859569              N/A    122.03m
   1   111.87         0.531876       63        0.0787678              N/A     45.53m
   2    85.69         0.498723       35        0.0497858              N/A     33.40m
   3    52.60         0.728958       35        0.0497858              N/A     18.81m
   4    32.38         0.941742       23        0.0497858              N/A     11.38m
   5    19.08         0.909782        5        0.0463943              N/A      7.25m
   6    23.00         0.475818        5        0.0463943              N/A      7.89m
   7    10.12         0.693365        5        0.0463943              N/A      4.20m
   8     6.53         0.586752        5        0.0463943              N/A  

In [19]:
# Modelo
model = SymbolicRegressor(population_size=7000,
                        init_depth=(5, 9),
                        generations=20, 
                        stopping_criteria=0.01,
                        const_range = (-1.0, 10.0),
                        p_crossover=0.9,
                        p_subtree_mutation=0.01, 
                        p_hoist_mutation=0.01, 
                        p_point_mutation=0.01, 
                        p_point_replace=0.05, 
                        max_samples=1.0,
                        verbose=1,
                        parsimony_coefficient=1e-3, 
                        random_state=0,
                        function_set=['add', 'sub', 'mul', 'div',pot_fn,pot2_fn],
                        #metric = rmsle, 
                        feature_names=['S', 'Re'])

model.fit(X_train, y_train)

# Obtener la expresión simbólica
expression = model._program
print("Expresión:", expression)

    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0   291.55       1.7173e+21      111         0.190911              N/A    134.68m
   1   137.68      2.62747e+07      111         0.190911              N/A     57.08m
   2   133.02      8.74897e+10       89         0.190911              N/A     54.67m
   3    69.74      6.47508e+11      115         0.150901              N/A     25.54m
   4    59.87           116388       31         0.109058              N/A     22.06m
   5    42.46      1.27483e+08       31         0.108969              N/A     13.64m
   6    35.42      9.08531e+10       27         0.107343              N/A     10.25m
   7    41.39      1.01484e+08       31         0.107343              N/A     11.97m
   8    36.43           294194       33         0.107325              N/A  

### Experimento: incorporar constantes cada una por separado

In [5]:
def pot(x1,B):
    con1 = x1 > 0
    con2 = (B > -1) & (B < 0)
    #con3 = (A>=1) & (A<8)
    with np.errstate(divide='ignore', invalid='ignore', over='ignore'):
        result = np.where(con1 & con2  ,np.power(x1,B), 0)
    return result

pot_fn= make_function(function=pot, name='pot', arity=2)

def pot2(x1,B):
    con1 = x1 > 0
    con2 = (B > -1) & (B < 0)
    #con3 = (A>=1) & (A<8)
    with np.errstate(divide='ignore', invalid='ignore', over='ignore'):
        result = np.where(con1 & con2  ,np.dot(2,np.power(x1,B)), 0)
    return result

pot2_fn= make_function(function=pot2, name='pot2', arity=2)

def pot3(x1,B):
    con1 = x1 > 0
    con2 = (B > -1) & (B < 0)
    #con3 = (A>=1) & (A<8)
    with np.errstate(divide='ignore', invalid='ignore', over='ignore'):
        result = np.where(con1 & con2  ,np.dot(3,np.power(x1,B)), 0)
    return result

pot3_fn= make_function(function=pot3, name='pot3', arity=2)

def pot4(x1,B):
    con1 = x1 > 0
    con2 = (B > -1) & (B < 0)
    #con3 = (A>=1) & (A<8)
    with np.errstate(divide='ignore', invalid='ignore', over='ignore'):
        result = np.where(con1 & con2  ,np.dot(4,np.power(x1,B)), 0)
    return result

pot4_fn= make_function(function=pot4, name='pot4', arity=2)

def pot5(x1,B):
    con1 = x1 > 0
    con2 = (B > -1) & (B < 0)
    #con3 = (A>=1) & (A<8)
    with np.errstate(divide='ignore', invalid='ignore', over='ignore'):
        result = np.where(con1 & con2  ,np.dot(5,np.power(x1,B)), 0)
    return result

pot5_fn= make_function(function=pot5, name='pot5', arity=2)

def pot6(x1,B):
    con1 = x1 > 0
    con2 = (B > -1) & (B < 0)
    #con3 = (A>=1) & (A<8)
    with np.errstate(divide='ignore', invalid='ignore', over='ignore'):
        result = np.where(con1 & con2  ,np.dot(6,np.power(x1,B)), 0)
    return result

pot6_fn= make_function(function=pot6, name='pot6', arity=2)

def pot7(x1,B):
    con1 = x1 > 0
    con2 = (B > -1) & (B < 0)
    #con3 = (A>=1) & (A<8)
    with np.errstate(divide='ignore', invalid='ignore', over='ignore'):
        result = np.where(con1 & con2  ,np.dot(7,np.power(x1,B)), 0)
    return result

pot7_fn= make_function(function=pot7, name='pot7', arity=2)

def pot8(x1,B):
    con1 = x1 > 0
    con2 = (B > -1) & (B < 0)
    #con3 = (A>=1) & (A<8)
    with np.errstate(divide='ignore', invalid='ignore', over='ignore'):
        result = np.where(con1 & con2  ,np.dot(8,np.power(x1,B)), 0)
    return result

pot8_fn= make_function(function=pot8, name='pot8', arity=2)

In [6]:
# Modelo
model = SymbolicRegressor(population_size=7000,
                        init_depth=(5, 9),
                        generations=20, 
                        stopping_criteria=0.01,
                        const_range = (-1.0, 10.0),
                        p_crossover=0.9,
                        p_subtree_mutation=0.01, 
                        p_hoist_mutation=0.01, 
                        p_point_mutation=0.01, 
                        p_point_replace=0.05, 
                        max_samples=1.0,
                        verbose=1,
                        parsimony_coefficient=1e-3, 
                        random_state=0,
                        function_set=['add', 'sub', 'mul', 'div',pot_fn,pot2_fn,pot3_fn, pot4_fn, pot5_fn, pot6_fn, pot7_fn, pot8_fn],
                        metric = rmsle, 
                        feature_names=['S', 'Re'])

model.fit(X_train, y_train)

# Obtener la expresión simbólica
expression = model._program
print("Expresión:", expression)

    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0   328.86          1.03025      255         0.107091              N/A    235.77m
   1   147.02         0.547944      133         0.107091              N/A     89.45m
   2    63.82         0.442465       39         0.100068              N/A     36.92m
   3    67.96         0.279877       45        0.0963336              N/A     36.87m
   4    42.86         0.354817       37        0.0862953              N/A     23.74m
   5    30.22         0.417715       15        0.0509017              N/A     16.00m
   6    21.12         0.547346       15        0.0509017              N/A     10.79m
   7    18.34         0.560621       17        0.0509017              N/A      8.69m
   8    14.52          0.62582       45        0.0493318              N/A  

In [8]:
# Modelo
model = SymbolicRegressor(population_size=7000,
                        init_depth=(5, 9),
                        generations=20, 
                        stopping_criteria=0.01,
                        const_range = (-1.0, 10.0),
                        p_crossover=0.9,
                        p_subtree_mutation=0.01, 
                        p_hoist_mutation=0.01, 
                        p_point_mutation=0.01, 
                        p_point_replace=0.05, 
                        max_samples=1.0,
                        verbose=1,
                        parsimony_coefficient=1e-2, 
                        random_state=0,
                        function_set=['add', 'sub', 'mul', 'div',pot_fn,pot2_fn,pot3_fn, pot4_fn, pot5_fn, pot6_fn, pot7_fn, pot8_fn],
                        metric = rmsle, 
                        feature_names=['S', 'Re'])

model.fit(X_train, y_train)

# Obtener la expresión simbólica
expression = model._program
print("Expresión:", expression)

    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0   328.86          1.03025      255         0.107091              N/A    210.09m
   1    53.99         0.826497        3         0.102646              N/A     33.85m
   2    31.63         0.611389        5        0.0911846              N/A     19.75m
   3     7.74          0.80168        3        0.0537202              N/A      5.81m
   4     4.03         0.499724        3        0.0537202              N/A      3.73m
   5     2.81         0.237125        3        0.0537202              N/A      3.64m
   6     2.57         0.205542        3        0.0537202              N/A      2.75m
   7     3.09         0.193897        3        0.0537202              N/A      2.01m
   8     3.07         0.190806        3        0.0537202              N/A  

In [7]:
# Modelo
model = SymbolicRegressor(population_size=7000,
                        init_depth=(5, 9),
                        generations=20, 
                        stopping_criteria=0.01,
                        const_range = (-1.0, 10.0),
                        p_crossover=0.9,
                        p_subtree_mutation=0.01, 
                        p_hoist_mutation=0.01, 
                        p_point_mutation=0.01, 
                        p_point_replace=0.05, 
                        max_samples=1.0,
                        verbose=1,
                        parsimony_coefficient=1e-3, 
                        random_state=0,
                        function_set=['add', 'sub', 'mul', 'div',pot_fn,pot2_fn,pot3_fn, pot4_fn, pot5_fn, pot6_fn],
                        metric = rmsle, 
                        feature_names=['S', 'Re'])

model.fit(X_train, y_train)

# Obtener la expresión simbólica
expression = model._program
print("Expresión:", expression)

    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0   319.27          1.04041        3        0.0746335              N/A    195.19m
   1   141.13         0.513472        3        0.0746335              N/A     80.32m
   2    68.45         0.418659        3        0.0628976              N/A     36.78m
   3    63.36         0.309766        3        0.0628976              N/A     29.01m
   4    31.14         0.574942        3        0.0628976              N/A     14.29m
   5    13.79         0.697558        5        0.0578128              N/A      7.43m
   6     5.74         0.424469        5        0.0578128              N/A      4.04m
   7     3.26         0.202009        5        0.0578128              N/A      2.22m
   8     3.50         0.204968        7        0.0468335              N/A  

### Experimento: incorporar el logaritmo del coseno hiperbolico del error como fitness

In [4]:
# x ^ B

def pot(x1, B):
    con1 = x1 > 0
    con2 = (B > -1) & (B < 0)
    #con3 = (A>=1) & (A<10)
    with np.errstate(divide='ignore', invalid='ignore', over='ignore'):
        result = np.where(con1 & con2 , np.power(x1,B), 0)
    return result

pot_fn= make_function(function=pot, name='pot', arity=2)

# A * x1 ^ B
def pot2(x1, A, B):
    con1 = x1 > 0
    con2 = (B > -1.5) & (B < 0)
    con3 = (A>1) & (A<8)
    with np.errstate(divide='ignore', invalid='ignore', over='ignore'):
        result = np.where(con1 & con2 & con3 , np.dot(A,np.power(x1,B)), 0)
    return result

pot2_fn= make_function(function=pot2, name='pot2', arity=3)

$
L_{log-cosh} =  \sum_{i} log(cosh((x_i - \hat{x}_i)))
$

In [5]:
def _logcosh(y, y_pred, w):
    """Calculate the log cos h."""
    #log_true = np.log((np.maximum(0.001, y) + 1))
    #log_pred = np.log((np.maximum(0.001, y_pred) + 1))
    loss = y_pred - y
    
    # Calcular el error cuadrático medio logarítmico
    #log_value = np.log(np.cosh(loss))
    
    return np.mean(np.log(np.cosh(loss)))

logcosh = make_fitness(function=_logcosh,
                    greater_is_better=False)

In [6]:
# Modelo
model = SymbolicRegressor(population_size=5000,
                        generations=20, 
                        stopping_criteria=0.01,
                        p_crossover=0.7, 
                        p_subtree_mutation=0.1,
                        p_hoist_mutation=0.05, 
                        p_point_mutation=0.1,
                        max_samples=0.9, 
                        verbose=1,
                        parsimony_coefficient=0.001,
                        const_range= (-1.0,8.0),
                        random_state=0,
                        function_set=['add', 'sub', 'mul', 'div',pot_fn, pot2_fn],
                        metric = logcosh, 
                        feature_names=['S', 'Re'])

model.fit(X_train, y_train)

# Obtener la expresión simbólica
expression = model._program
print("Expresión:", expression)

# prediciendo con el conjunto de prueba

y_pred = model.predict(X_test)

# Calcular métricas
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)


print("MSE:", mse)
print("R^2:", r2)
print("MAE:", mae)
print("MAPE:",mape*100)

    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left


C:\Users\lucas\AppData\Local\Temp\ipykernel_13408\938827838.py:10: RuntimeWarning: overflow encountered in cosh
  return np.mean(np.log(np.cosh(loss)))


   0    56.37              inf       50         0.037821         0.037821     41.18m
   1    34.48              inf      178        0.0169477        0.0169477     24.13m
   2    47.59              nan       27              nan              nan     30.85m
   3    17.49              nan       31              nan              nan     12.37m
   4    10.82              nan       27              nan              nan     10.62m
   5    16.90              nan       27              nan              nan     10.47m
   6    32.30              nan       30              nan              nan     16.69m
   7    36.93              nan       35              nan              nan     15.44m
   8    42.05              nan       37              nan              nan     16.63m
   9    47.56              nan       23              nan              nan     15.88m
  10    53.33              nan       71              nan              nan     18.36m
  11    59.69              nan       48              nan         

ValueError: Input contains NaN.

In [24]:

y_test_ = y_test.values.flatten()
y_test_

array([2.12230538, 2.20442975, 2.42085352, ..., 1.52151373, 1.52192079,
       1.43975326])

In [25]:
y_test_ - y_pred

array([-0.15197103, -0.06984666,  0.14648858, ..., -0.15260832,
       -0.15220641, -0.23438012])

In [26]:
np.cosh(y_test_ - y_pred)

array([1.01156984, 1.00244027, 1.01074865, ..., 1.01166727, 1.01160578,
       1.02759299])

In [29]:
np.log(np.cosh(10*(y_test_ - y_pred)))

array([0.08733156, 0.02263442, 0.08237704, ..., 0.08791098, 0.08754549,
       0.16598206])

In [39]:
import math

np.sum(np.log(math.cosh((y_pred - y_test_))))

TypeError: only length-1 arrays can be converted to Python scalars